# Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Part 1

Use the Notebook to build the code to scrape [this Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

__Get HTML Data from Webpage__

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_source = requests.get(url).text

In [3]:
soup = BeautifulSoup(url_source, 'html.parser')

__Extract Table from Data__

In [4]:
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

__Process Data to Remove Values Not Assigned__

In [5]:
result = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    if row != [] and row[1] != "Not assigned\n":
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        result.append(row)

__Create DataFrame__

In [6]:
df = pd.DataFrame(result, columns = ["PostalCode", "Borough", "Neighborhood"])
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


__Group All Neighborhoods with Same Zipcode__

In [7]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


__Print Number of Rows in DataFrame__

In [8]:
print("# Rows: ", df.shape)

# Rows:  (103, 3)


## Part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

**Get CSV Data**

In [9]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge Latitude/Longitude Data with Previous Dataframe**

In [10]:
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_df_merged = pd.merge(geo_df, df, on='PostalCode')

geo_df_data = geo_df_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_df_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3

In [11]:
CLIENT_ID = "1UPBDV2AHTPBWVQXPY0QMT5PRVZU2353V41XFCAX1JS0EA5H" #My Foursquare client ID
CLIENT_SECRET = "QYBZFUD3LJHKVCPOELBYWI5ERNCX4FJTBVRFHDT4YDGN2JVB" #Your Foursquare Client Secret
VERSION = '20180605'

In [12]:
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 1UPBDV2AHTPBWVQXPY0QMT5PRVZU2353V41XFCAX1JS0EA5H
CLIENT_SECRET:QYBZFUD3LJHKVCPOELBYWI5ERNCX4FJTBVRFHDT4YDGN2JVB


In [13]:
toronto_only_df = geo_df_data[geo_df_data['Borough'].str.contains('Toronto', regex=False)]
toronto_only_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


**Vizualizing All Neighborhoods**

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed.')

Solving environment: done

# All requested packages already installed.

Folium installed.


In [15]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(toronto_only_df['Latitude'],
                                         toronto_only_df['Longitude'], toronto_only_df['Borough'],
                                         toronto_only_df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

**Cluster the Data Using K-Means Clustering**

In [16]:
from sklearn.cluster import KMeans
print('KMeans imported.')

KMeans imported.


In [17]:
if 'Cluster Labels' in toronto_only_df.columns:
    del toronto_only_df['Cluster Labels']
    
k = 5
clustering_data = toronto_only_df.drop(['PostalCode','Borough','Neighborhood'],1)
k_means = KMeans(n_clusters = k, random_state=0).fit(clustering_data)
print(k_means.labels_)
toronto_only_df.insert(0, 'Cluster Labels', k_means.labels_)

[4 4 4 4 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 3 3 3 1 1 1 3 0 3 3 0 0 0
 1 4]


In [18]:
toronto_only_df

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,4,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,4,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,2,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,2,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,2,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,2,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


**Vizualizing Clustered Neighborhoods**

In [19]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [20]:
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_only_df['Latitude'],
                                            toronto_only_df['Longitude'], toronto_only_df['Neighborhood'],
                                            toronto_only_df['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters